In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Pretenders,Back On The Chain Gang,"I found a picture of you, oh oh oh oh \r\nWha..."
1,Def Leppard,Have You Ever Needed Someone So Bad,"Here I am, I'm in the wrong bed again \r\nIt'..."
2,Morrissey,Oh Phoney,May this lovely letter \r\nReach its destinat...
3,Carly Simon,Big Dumb Guy,So many tech-heads \r\nRecent invasion \r\nI...
4,Patsy Cline,Foolin' Around,I know that you've been foolin' around on me r...
5,Clash,Justice Tonight-Kick It Over,Stay around don't play around \r\nThis old to...
6,Stevie Ray Vaughan,Tightrope,Runnin' through this business of life \r\nRai...
7,Christina Aguilera,Choices Made,"All i get is mixed signals from you, \r\nBut ..."
8,John Mellencamp,Now More Than Ever,If you believe \r\nWon't you please raise you...
9,Utopia,Lysistrata,"Lysistrata, open up the bedroom door \r\nWhat..."


In [9]:
df['text'][0]

"I found a picture of you, oh oh oh oh  \r\nWhat hijacked my world that night  \r\nTo a place in the past  \r\nWe've been cast out of? Oh oh oh oh  \r\nNow we're back in the fight  \r\nWe're back on the train  \r\nOh, back on the chain gang  \r\n  \r\nA circumstance beyond our control, oh oh oh oh  \r\nThe phone, the TV and the news of the world  \r\nGot in the house like a pigeon from hell, oh oh oh oh  \r\nThrew sand in our eyes and descended like flies  \r\nPut us back on the train  \r\nOh, back on the chain gang  \r\n  \r\nThe powers that be  \r\nThat force us to live like we do  \r\nBring me to my knees  \r\nWhen I see what they've done to you  \r\nBut I'll die as I stand here today  \r\nKnowing that deep in my heart  \r\nThey'll fall to ruin one day  \r\nFor making us part  \r\n  \r\nI found a picture of you, oh oh oh oh  \r\nThose were the happiest days of my life  \r\nLike a break in the battle was your part, oh oh oh oh  \r\nIn the wretched life of a lonely heart  \r\nNow we'r

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
# nltk.download()
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def token(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: token(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.03062669, 0.23418482, ..., 0.00676618, 0.01158039,
       0.00939837])

In [21]:
df[df['song'] == 'Oh Phoney']

,artist,song,text
2,Morrissey,Oh Phoney,may thi love letter reach it destin if onli qu...


In [26]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:6]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [27]:
recommendation('Oh Phoney')

['Million Dollar Bill',
 'Back On The Chain Gang',
 'Pavlove',
 'Oh Well',
 'Oh Oh I Love Her So']

In [28]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))